# Period Module

## Initialisation

Import modules, etc

In [1]:
import os
import operator
import jinja2
import re

from datetime import datetime

import unittest

from common import Printable, testExit, projdir

from name import Name
from fuzzy import FuzzyMatch
from entrant import Entrant

from constants import *

## Period Class

The period class contains common methods for events, sessions and courses

In [2]:
class Period(Printable):
    def __init__(self, parent=None, verbosity=1):
        '''Initialise period object'''

        super().__init__(verbosity=verbosity)

        self.period = self.__class__.__name__
        self.parent = parent

        if parent:
            self.entrants = parent.entrants
            self.names = parent.names
            self.sailNos = parent.sailNos
            self.gt31Ids = parent.gt31Ids
            self.fuzzyMatch = parent.fuzzyMatch
            self.appConfig = parent.appConfig
            self.eventConfig = parent.eventConfig
            self.reports = parent.reports

        else:
            self.entrants = {}
            self.names = {}
            self.sailNos = {}
            self.gt31Ids = {}
            self.fuzzyMatch = FuzzyMatch()
            self.appConfig = None
            self.eventConfig = None
            self.reports = None

        self.runs = {}
        self.numRuns = 0


    def getNextEntrantId(self):
        '''Get the next entrant ID'''
        
        if len(self.entrants) > 0:
            entrantId = max(self.entrants) + 1
        else:
            entrantId = 1

        return entrantId


    def getEntrantBySailNo(self, sailNo, name):
        '''Get entrant from the sail number'''

        if sailNo not in self.sailNos:

            # Start by attempting a quick lookup of the name itself
            # TODO - match by craft type as well as name
            if name and name in self.names:
                entrant = self.names[name][0]
                self.sailNos[sailNo] = entrant
                self.logWarning('Auto-matched sail number {} to {} ({})'.format(
                    sailNo, entrant.getName(), entrant.getValue('Craft Type')))
               
            else:
                # Next try looking for entrant names that count as a fuzzy match
                # TODO - match by craft type as well as name
                entrants = []
                if name:
                    nameObj = Name(name)
                    for entrantId in self.entrants:
                        entrant = self.entrants[entrantId]
                        if self.fuzzyMatch.matchNameObjects(entrant.name, nameObj):
                            entrants.append(entrant)

                # Only accept a unique match - multiple matches will be ignored
                if len(entrants) == 1:
                    entrant = entrants[0]
                    self.names[name] = [entrant]
                    self.sailNos[sailNo] = entrant
                    self.logWarning('Auto-matched sail number {} to {} ({})'.format(
                        sailNo, entrant.getName(), entrant.getValue('Craft Type')))

                else:
                    entrantId = self.getNextEntrantId()

                    if name:
                        entrant = Entrant(
                            self.eventConfig, ["ID", "Sail Number", "Name"], [entrantId, sailNo, name],
                            verbosity=self.verbosity)
                        self.logWarning('Unrecognised sail number {} ({}) on {}'.format(sailNo, entrant.getName(), self.date))
                    else:
                        entrant = Entrant(
                            self.eventConfig, ["ID", "Sail Number"], [entrantId, sailNo], verbosity=self.verbosity)
                        self.logWarning('Unrecognised sail number {} on {}'.format(sailNo, self.date))

                    self.entrants[entrantId] = entrant
                    self.names[name] = [entrant]
                    self.sailNos[sailNo] = entrant

        else:
            entrant = self.sailNos[sailNo]

            # TODO - check craft type as well as name
            if name:
                if name != entrant.getName():
                    nameObj = Name(name)
                    if self.fuzzyMatch.matchNameObjects(nameObj, entrant.name) is False:
                        self.logWarning('Name mismatch for sail {} - {} vs {}'.format(sailNo, name, entrant.getName()))

                if name not in self.names:
                    self.names[name] = [entrant]

        return entrant


    def getEntrantByGt31(self, gt31Id, gt31Serial):
        '''Get entrant from the GT-31 ID and serial'''

        gt31Id = gt31Id.upper()

        if gt31Id not in self.gt31Ids:
            # Try to determine name elements from GT-31 ID
            reMatch = re.match('([A-Z]+)[1-9][0-9]*([A-Z]+)', gt31Id)
            if reMatch:
                lastName, firstName = reMatch.groups()
            else:
                lastName = firstName = 'XXXXX'

            # Try looking for entrant names that match the GT-31 ID
            entrants = []
            regex = re.compile('[^A-Z]')

            for entrantId in self.entrants:
                entrant = self.entrants[entrantId]
                entrantFirstName = entrant.getValue('First Name')
                entrantLastName = entrant.getValue('Family Name')

                if entrantFirstName and entrantLastName:
                    # Remove non-alphas from the name; spaces, hyphens, apostrophes, etc.
                    # Replicates SSERPENT logic which takes the first 5 letters prior to filtering
                    modifiedFirstName = regex.sub('', entrantFirstName[:5].upper())
                    modifiedLastName = regex.sub('', entrantLastName[:5].upper())

                    if (modifiedFirstName == firstName and modifiedLastName == lastName or
                            modifiedFirstName == lastName and modifiedLastName == firstName):
                        entrants.append(entrant)

            # Only accept a unique match - multiple matches will be ignored
            if len(entrants) == 1:
                entrant = entrants[0]
                name = entrant.getName()

                self.names[name] = [entrant]
                self.gt31Ids[gt31Id] = entrant

                self.logWarning('Auto-matched GT-31 ID {} to {} ({})'.format(
                    gt31Id, entrant.getName(), entrant.getValue('Craft Type')))

            else:
                self.logWarning('Unrecognised GT-31 ID {} on {}'.format(gt31Id, self.date))

                entrantId = self.getNextEntrantId()

                entrant = Entrant(
                    self.eventConfig, ["ID", "GT31 ID", "GT31 SN"], [entrantId, gt31Id, gt31Serial], verbosity=self.verbosity)
                name = entrant.getName()

                self.entrants[entrantId] = entrant
                self.names[entrant] = [entrant]
                self.gt31Ids[gt31Id] = entrant

        else:
            entrant = self.gt31Ids[gt31Id]

            # Only report unrecognised GT-31 serials if the entrant had any GT-31 serials registered
            if entrant.gt31SerialNumbers and gt31Serial not in entrant.gt31SerialNumbers:
                self.logWarning('Unrecognised GT-31 SN for {} ({}) on {} - {} vs {}'.format(
                        entrant.getValue('Name'), entrant.getValue('Craft Type'), self.date,
                        gt31Serial, entrant.gt31SerialNumbers))

                entrant.gt31SerialNumbers.add(gt31Serial)

        return entrant


    def storeRun(self, entrantId, speedRun):
        '''Store run in memory'''

        if entrantId in self.runs:
            self.runs[entrantId].append(speedRun)
        else:
            self.runs[entrantId] = [speedRun]
            
        self.numRuns += 1

        if self.parent:
            self.parent.storeRun(entrantId, speedRun)


    def sortRuns(self):
        '''Sort runs for each person, fastest to slowest'''

        entrants = []

        for entrantId in self.runs:
            # Sort the runs for the individual entrant
            runs = self.runs[entrantId]
            runs.sort(key=lambda x: x.data[T_SPEED], reverse=True)
            
            # Period sort will be based on speed descending (hence the negative) and date/time ascending
            bestRun = runs[0]
            entrants.append((entrantId, runs, -bestRun.speed, bestRun.date, bestRun.time))

        # Sort avoids using a lambda function call
        entrants.sort(key = operator.itemgetter(2, 3, 4))
        
        # Re-create self.runs so that it is correctly sorted (Python 3.7 upwards)
        self.runs = {}
        for entrant in entrants:
            self.runs[entrant[0]] = entrant[1]


    def getEventName(self):
        '''Get the event name from event config or app config'''

        eventName = None

        if 'Event' in self.eventConfig:
            if 'Name' in self.eventConfig['Event']:
                eventName = self.eventConfig['Event']['Name']

        if eventName is None and 'Event' in self.appConfig:
            if 'Name' in self.appConfig['Event']:
                eventName = self.appConfig['Event']['Name']

        if eventName is None:
            eventName = 'Weymouth Speed Week'

        return eventName


    def runReports(self):
        '''Run all of the reports results'''

        numWarnings = 0

        # Note: It is slightly debatable whether this class should interact with the Reports class at such a low level
        className = self.__class__.__name__
        if className in self.reports.periods:
            reportSets = self.reports.periods[className]

            templatePath = os.path.join(projdir, 'python', 'templates')
            templateLoader = jinja2.FileSystemLoader(searchpath=templatePath)
            templateEnv = jinja2.Environment(loader=templateLoader, autoescape=True, trim_blocks=True, lstrip_blocks=True)

            eventName = self.getEventName()
            if className == 'Session':
                periodDate = datetime.strptime(self.date, "%Y%m%d").strftime("%a %-d %b")
                pageComment = self.comment
            else:
                periodDate = self.year
                pageComment = None

            for reportSet in reportSets.keys():
                reports = reportSets[reportSet]
                numResults = 0

                for report in reports:
                    report.prepareResults(self.runs)
                    numResults += len(report.results)

                    if report.warning and len(report.results) > 0:
                        numWarnings += 1

                htmlDir = os.path.join(projdir, 'docs', 'results', str(self.year))
                if className == 'Session':
                    htmlDir = os.path.join(htmlDir, self.date)
                if not os.path.exists(htmlDir):
                    os.makedirs(htmlDir)

                htmlFile = os.path.join(htmlDir, reportSet + '.html')

                if numResults > 0 or pageComment:
                    template = templateEnv.get_template("results.html")
                    html = template.render(
                                    eventName=eventName,
                                    className=className,
                                    pageTitle=reports[0].suptitle,
                                    pageComment=pageComment,
                                    pageDescription=reports[0].suptitle,
                                    periodDate=periodDate,
                                    cssPath='../css',
                                    reports=reports)

                    with open(htmlFile, 'w', encoding='utf-8') as f:
                        f.write(html)
                else:
                    try:
                        os.remove(htmlFile)
                    except OSError:
                        pass
                    
        if numWarnings > 0:
            if className == 'Session':
                self.logWarning('{} report on {} generated results - {}'.format(
                    className, self.date, report.filter))
            else:
                self.logWarning('{} report generated results - {}'.format(
                    className, report.filter))

## Unit Tests

A handful of very basic tests, including a dummy session class

In [3]:
class TestPeriod(unittest.TestCase):
    '''Class to test Period class'''
    
    def testDummy(self, session=None):
        '''Test using dummy data'''

        pass

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [4]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## All Done!